In [ ]:
# ==== CELL 0: CONFIG & DEVICE ====
import os, torch

# Đường dẫn dữ liệu (đúng cấu trúc Train/Validate/Test như bạn dùng)
ROOT = r"C:/TRAIN/Deep Learning/vietnamese-foods/Images"
root_test = f"{ROOT}/Test"

IMG_SIZE = 224
BATCH_SIZE = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# ==== CELL 1: TRANSFORMS GIỐNG LÚC TRAIN ====
from torchvision import transforms

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    # Nếu lúc train có Normalize(mean, std) thì PHẢI dùng y chang:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225]),
])

In [45]:
# ==== CELL 2: HÀM BUILD + GỌI HÀM => TẠO test_loader, class_names ====
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

def build_test_loader(root_test_dir, transform, batch_size):
    dataset = ImageFolder(root=root_test_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    class_names = list(dataset.classes)            # thứ tự theo thư mục của ImageFolder
    class_to_idx = dict(dataset.class_to_idx)      # mapping {class_name: idx} của dataset
    return loader, class_names, class_to_idx

# >>> GỌI HÀM (rất hay bị quên)
test_loader, class_names, ds_class_to_idx = build_test_loader(root_test, test_transform, BATCH_SIZE)

print("Số lớp:", len(class_names))
print("Ví dụ lớp:", class_names[:33])

Số lớp: 33
Ví dụ lớp: ['Banh beo', 'Banh bot loc', 'Banh can', 'Banh canh', 'Banh chung', 'Banh cuon', 'Banh duc', 'Banh gio', 'Banh khot', 'Banh mi', 'Banh pia', 'Banh tet', 'Banh trang nuong', 'Banh xeo', 'Bun bo Hue', 'Bun dau mam tom', 'Bun mam', 'Bun rieu', 'Bun thit nuong', 'Ca kho to', 'Canh chua', 'Cao lau', 'Chao long', 'Com tam', 'Goi cuon', 'Hu tieu', 'Mi quang', 'Nem chua', 'Pho', 'Xoi xeo', 'banh_da_lon', 'banh_tieu', 'banh_trung_thu']


In [ ]:
# ==== CELL 3: EVALUATION CORE – fixed label order & stable CM ====
import numpy as np, torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

# 1) Chốt thứ tự nhãn (đã có class_names từ Cell 2)
assert "class_names" in globals(), "Bạn cần nạp class_names (thứ tự chuẩn) trước Cell 3!"
NUM_CLASSES = len(class_names)
name2idx = {n: i for i, n in enumerate(class_names)}

# 2) Tạo remap nếu dataset.class_to_idx không trùng thứ tự class_names
remap = None
if hasattr(test_loader.dataset, "class_to_idx"):
    ds_idx2name = {v: k for k, v in test_loader.dataset.class_to_idx.items()}
    # hoặc dùng ds_class_to_idx từ Cell 2:
    # ds_idx2name = {v: k for k, v in ds_class_to_idx.items()}
    remap = {src_idx: name2idx[ds_idx2name[src_idx]] for src_idx in range(len(ds_idx2name))}

def collect_logits(model, loader, device, remap=None):
    y_true, y_pred, y_prob = [], [], []
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            if remap is not None:
                y = torch.as_tensor([remap[int(t)] for t in y], dtype=torch.long)
            x = x.to(device)
            logits = model(x)
            prob = torch.softmax(logits, dim=1).cpu().numpy()
            pred = prob.argmax(1)

            y_prob.append(prob)
            y_pred.append(pred)
            y_true.append(y.numpy())
    return np.concatenate(y_true), np.concatenate(y_pred), np.concatenate(y_prob)

# Sau Cell 3, bạn có thể dùng tiếp các cell đánh giá, vẽ CM, v.v.
print("OK: Core evaluation đã sẵn sàng.")

OK: Core evaluation đã sẵn sàng.


In [47]:
# QUICK CHECK trước khi vẽ
try:
    _ = iter(test_loader)
    assert len(class_names) > 0
    print("✅ test_loader & class_names sẵn sàng.")
except Exception as e:
    print("❌ Lỗi:", e)


✅ test_loader & class_names sẵn sàng.
